### **Imports**

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
# from scipy.sparse import random
# from scipy import stats
# import seaborn as sns
import matplotlib.pyplot as plt
# import plotly.express as px
# from sklearn.model_selection import train_test_split
import geopandas as gpd
import shapely
import shapely.wkt
from geomet import wkt

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
dfCapi= pd.read_csv("parte1.csv")
barrios=pd.read_csv("barrios.csv")

In [ ]:
#Armamos una pivot con los place_name de nuestro data frame para checkear si matchean con el csv barrios
dfCapi_grouped=dfCapi.groupby('place_name').mean()
dfCapi_grouped

In [ ]:
geoprop = gpd.GeoDataFrame(dfCapi, geometry=gpd.points_from_xy(dfCapi.lon, dfCapi.lat))
barrios["WKT_shapely"] = barrios["WKT"].apply(shapely.wkt.loads) 
geo_barrios = gpd.GeoDataFrame(barrios, geometry='WKT_shapely')

In [ ]:
# Funcion para obtener el BARRIO segun las coordenadas geográficas de la propiedad
def barrio_gpd(x):
    if ((x.place_name == "Capital Federal") & (np.isnan(x.lat) == False)): # Solo busco los que tienen place_name = Capital Federal y valores de lat-lon
        for barrio in geo_barrios.barrio:
            r = geo_barrios[geo_barrios.barrio == barrio].contains(x.geometry).values[0] #Itero sobre todos los barrios y me fijo cual contiene al punto
            if (r):
                return barrio #Devuelvo el nombre del barrio que contiene el punto
    else: 
        return x.place_name # Si place_name != Capital Federal o no tenia coordenadas devuelvo el valor original

In [ ]:
# Reemplazo el barrio por gps en place_name
dfCapi.loc[:,"place_name"] = geoprop.apply(barrio_gpd, axis=1)

In [ ]:
import geopy.distance
df_subte = pd.read_csv("./estaciones-de-subte.csv", sep=',')
geo_subte = gpd.GeoDataFrame(df_subte,geometry=gpd.points_from_xy(df_subte.long,df_subte.lat))
geoprop.loc[:,"distancia_subte"] = 10000 #seteo un valor por defecto para poder empezar comparando

In [ ]:
# SE DEJA COMENTADA ESTA LINEA PORQUE TARDA MUCHO. EL RESULTADO DE ESTA ITERACION YA SE ENCUENTRA EN EL ARCHIVO parte2.csv
# for i in geo_subte.index: #itero sobre cada estacion
#     for j in geoprop.index: #itero sobre cada casa
#         if (geoprop.iloc[j].geometry.x == geoprop.iloc[j].geometry.x): #condicion rara para chequear si el valor no es None
#             dist = geopy.distance.geodesic((geo_subte.loc[i,"geometry"].x,geo_subte.loc[i,"geometry"].y),
#                                            (geoprop.loc[j,'geometry'].x,geoprop.loc[j,'geometry'].y)).meters # calculo la distancia
#             if (dist < geoprop.distancia_subte[j]): #chequeo que sea menor a la registrada
#                 geoprop.loc[j, 'distancia_subte'] = dist #si lo es, le digo que esa sea la nueva distancia

In [ ]:
geoprop.distancia_subte.replace(10000,np.NaN, inplace=True) #reemplazo los casos que quedaron con el valor inicial arbitrario por null
geoprop.distancia_subte = geoprop.distancia_subte.round(2)
dfCapi["distancia_subte"] = geoprop.distancia_subte

In [16]:
# dfCapi.to_csv("parte2.csv")
# barrios.to_csv("barrios.csv")